<a href="https://colab.research.google.com/github/BKousha/FloorPlanSegmentation/blob/main/training_TF_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D,Conv2DTranspose

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [ ]:
# # Define the class mapping for merging
# class_names = {
#     0: "background",
#     1: "room",
#     2: "hallway",
#     3: "kitchen",
#     4: "columns",
#     5: "mep",
#     6: "windows",
#     7: "doors",
#     8: "glass_doors",
#     9: "bathrooms",
#     10: "elevators",
#     11: "elevator_doors",
#     12: "stairs",
#     13: "storage",
#     14: "electrical"
# }
# # class_mapping  = {
# #     0:  [0],   # Background
# #     1:  [1],   # Room
# #     2:  [2],   # Hallway
# #     3:  [3],   # Kitchen
# #     4:  [4],   # Columns
# #     5:  [6],   # windows
# #     6:  [7,8], # Doors,Glass Doors
# #     7:  [9],   # Bathrooms
# #     8:  [11],  # elevator_doors
# #     9:  [5, 10, 12, 13],  # MEP, Elevators, Stairs, Storage
# #     10: [14]   # Electrical
# # }
# def translate_array(arr):
#     # Mapping dictionary for category translation
#     translation_dict = {
#         5: 9,   # Merge MEP into Bathrooms
#         10: 9,  # Merge Elevators into Bathrooms
#         12: 9,  # Merge Stairs into Bathrooms
#         13: 9,  # Merge Storage into Bathrooms
#         7: 8,   # Merge Doors into Glass Doors
#     }

#     # Create a new array by mapping the original array using the translation dictionary
#     translated_arr = np.vectorize(translation_dict.get)(arr)

#     return translated_arr

In [ ]:
# Set the necessary parameters
image_dir = "/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/temp_images"
mask_dir = "/content/drive/MyDrive/Processed_Floor_Plans/smaller_images/temp_masks"
batch_size = 8
target_size = (512, 512)
num_classes = 10

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

# Load the image and mask files
image_files = []
mask_files = []
for folder_name in ['1', '2', '4', '8']:
    folder_image_dir = os.path.join(image_dir, folder_name)
    folder_mask_dir = os.path.join(mask_dir, folder_name)

    image_files.extend(sorted(listdir_fullpath(folder_image_dir)))
    mask_files.extend(sorted(listdir_fullpath(folder_mask_dir)))

print(len(image_files))
print(len(mask_files))



24903
24903


In [ ]:
import cv2
from tqdm import tqdm
threshold=2000
class_counts = {class_name: 0 for class_name in class_names.values()}

for mask_file in tqdm(mask_files):
    # Load the mask image

    mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)

    # Calculate the count of each class in the mask image
    for class_value, class_name in class_names.items():
        class_pixels = np.sum(mask == class_value)
        if class_pixels >= threshold:
            class_counts[class_name] += 1


total_files = len(mask_files)
class_percentages = {class_name: (count / total_files) * 100 for class_name, count in class_counts.items()}

# Print the results
for class_name, percentage in class_percentages.items():
    print(f"Class: {class_name} - Percentage: {percentage}%")

 32%|███▏      | 7982/24903 [56:30<2:05:44,  2.24it/s]

In [ ]:
class SegmentationDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_files, mask_files, batch_size, target_size, num_classes):
        self.image_files = image_files
        self.mask_files = mask_files
        #self.image_dir = image_dir
        #$self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.target_size = target_size
        self.num_classes = num_classes
        self.indexes = np.arange(len(self.image_files))

    def __len__(self):
        return len(self.image_files) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images = []
        batch_masks = []

        for i in batch_indexes:
            image_file = self.image_files[i]
            mask_file = self.mask_files[i]

            #image_path = os.path.join(self.image_dir, image_file)
            #mask_path = os.path.join(self.mask_dir, mask_file)

            image = tf.keras.preprocessing.image.load_img(image_file, color_mode="rgb", target_size=self.target_size)
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = image / 255.0

            mask = tf.keras.preprocessing.image.load_img(mask_file, color_mode="grayscale", target_size=self.target_size)
            mask = tf.keras.preprocessing.image.img_to_array(mask)
            mask = self.preprocess_mask(mask)

            batch_images.append(image)
            batch_masks.append(mask)

        return np.array(batch_images), np.array(batch_masks)

    def preprocess_mask(self, mask):
        merged_mask = np.zeros(mask.shape[:2])

        for new_class, old_classes in class_mapping.items():
            merged_mask[np.isin(mask, old_classes)] = new_class

        return to_categorical(merged_mask, num_classes=self.num_classes)


In [ ]:
# Split the files into training and validation sets
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Create the data generators for training and validation
train_data_generator = SegmentationDataGenerator(train_files, mask_files, image_dir, mask_dir, batch_size, target_size, num_classes)
val_data_generator = SegmentationDataGenerator(val_files, mask_files, image_dir, mask_dir, batch_size, target_size, num_classes)


# Retrieve one sample from the validation data generator
sample_index = 0  # Choose the index of the sample you want to visualize
sample_image, sample_mask = val_data_generator[sample_index]


# Plot the original image and the predicted mask
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(sample_image[0])
axes[0].set_title('Original Image')
axes[0].axis('off')
axes[1].imshow(sample_mask[0], cmap='jet', vmin=0, vmax=num_classes-1)
axes[1].set_title('Mask')
axes[1].axis('off')

# Create a colorbar legend for the predicted mask
cmap = plt.cm.get_cmap('jet', num_classes)
cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax=axes[1], ticks=np.arange(num_classes))
#cbar.set_ticklabels(labels)

plt.tight_layout()
plt.show()


IndexError: ignored

In [ ]:


# Make predictions on the sample using the trained model
predictions = model.predict(sample_image)

# Plot the original image and the predicted mask
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(sample_image[0])
axes[0].set_title('Original Image')
axes[0].axis('off')
axes[1].imshow(np.argmax(predictions[0], axis=-1), cmap='jet', vmin=0, vmax=num_classes-1)
axes[1].set_title('Predicted Mask')
axes[1].axis('off')

# Create a colorbar legend for the predicted mask
cmap = plt.cm.get_cmap('jet', num_classes)
cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap), ax=axes[1], ticks=np.arange(num_classes))
cbar.set_ticklabels(labels)

plt.tight_layout()
plt.show()


IndexError: ignored

In [ ]:


def unet(nclass):
    # Input layer
    inputs = Input(shape=(None, None, 3))

    # Contracting path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expanding path
    up6 = Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    concat6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(concat6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    concat7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(concat7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    concat8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(concat8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    concat9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(concat9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer
    outputs = Conv2D(nclass, 1, activation='softmax')(conv9)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    return model

# Define the number of output classes
nclass = 10


# Build the model
input_shape = (512, 512, 3)
model = unet(num_classes)

# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_4 (Conv2D)              (None, None, None,   1792        ['input_2[0][0]']                
                                64)                                                               
                                                                                                  
 conv2d_5 (Conv2D)              (None, None, None,   36928       ['conv2d_4[0][0]']               
                                64)                                                           

In [ ]:

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

# Train the model using the data generators
history=model.fit(train_data_generator, epochs=10, validation_data=val_data_generator)


FileNotFoundError: ignored

In [ ]:
# Load the best saved model
best_model = tf.keras.models.load_model("best_model.h5")

# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_data_generator)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)


In [ ]:
# Get the loss values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss values
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'g', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Loss Function')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()